In [3]:
import random

from deap import base
from deap import creator
from deap import tools

In [2]:
import sys
sys.path.insert(1, '/path/to/Wind Farm Evaluator')
from Vec_modified import *

def get_args(n_turbs):
    # setting turbine radius
    turb_rad = 50.0

    # Loading the power curve
    power_curve   =  loadPowerCurve('C:/Users/awals/Downloads/Shell AI Hackathon/Shell_Hackathon Dataset/power_curve.csv')

    # Loading wind data 
    years = ['07','08','09','13','14','15','17']
    wind_inst_freqs = []
    for y in years:
        wind_inst_freqs.append(binWindResourceData(f'C:/Users/awals/Downloads/Shell AI Hackathon/Shell_Hackathon Dataset/Wind Data/wind_data_20{y}.csv'))
    
    # preprocessing the wind data to avoid repeated calculations
    n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t = preProcessing(power_curve,n_turbs)

    kwargs = {'turb_rad': turb_rad, 'power_curve': power_curve, 'wind_inst_freqs': wind_inst_freqs,
            'n_wind_instances': n_wind_instances, 'cos_dir': cos_dir, 'sin_dir': sin_dir, 'wind_sped_stacked': wind_sped_stacked,
            'C_t': C_t}

    return kwargs

def proxi_constraint(turb_coords):
    """
        Function to penalize if proximity contraint is violated.
        turb_coords is a 2d numpy array with N (xi,yi) elements.
    """
    ans = 0
    for i in range(turb_coords.shape[0]-1):
        for j in range(i+1,turb_coords.shape[0]-1):
            norm = np.linalg.norm(turb_coords[i]-turb_coords[j])
            ans += max(0,400 - norm)
            
    return ans


def evaluate(individual, n_turbs, **kwargs):
    """
        Objective function to be minimized w.r.t. turb_coords.
    """

    turb_coords = np.array([[individual[i],individual[i+1]] for i in range(0,2*n_turbs-1,2)])
    mean_AEP = 0
    for wind_inst_freq in wind_inst_freqs:
        mean_AEP += getAEP(turb_rad, turb_coords, power_curve, wind_inst_freq, n_wind_instances, cos_dir, sin_dir, wind_sped_stacked, C_t)
    
    mean_AEP /= 7
    
    proxi_val = proxi_constraint(turb_coords)
    
    ideal_AEP = 11.297*n_turbs  # 11.297 is the mean score for 1 turbine
    
    return mean_AEP/ideal_AEP, proxi_val

In [4]:
import array
creator.create("FitnessMulti", base.Fitness, weights=(1.0,-1.0))
creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMulti, strategy=None)
creator.create("Strategy", array.array, typecode="d")

FLT_MIN = 50
FLT_MAX = 3950
N_DIM = 100  # 50*2
smin, smax = 1.0, 20.0  # min and max standard deviations

def initES(icls, scls, n, FLT_MIN, FLT_MAX, smin, smax):
    ind = icls(random.uniform(FLT_MIN, FLT_MAX) for _ in range(n))
    ind.strategy = scls(random.uniform(smin,smax) for _ in range(n))

    return ind

toolbox = base.Toolbox()

toolbox.register("attr_float", random.uniform, FLT_MIN, FLT_MAX)
toolbox.register("individual", initES, creator.Individual, creator.Strategy, N_DIM, FLT_MIN, FLT_MAX, smin, smax)

In [5]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [11]:
N_CHILD = 50
N_PARENT = 5
Offspring = toolbox.population(n=N_CHILD)

100